In [ ]:
!conda install -y gdown

In [ ]:
!gdown https://github.com/shibhansh/loss-of-plasticity/archive/refs/heads/main.zip

In [ ]:
!echo "A" | unzip /kaggle/working/main.zip -d /kaggle/working/
!mkdir /kaggle/working/lop
!mkdir /kaggle/working/data/
!mv /kaggle/working/loss-of-plasticity-main/* /kaggle/working/lop/

In [ ]:
import sys
import json
import torch
import pickle
import argparse
import numpy as np
import matplotlib.pyplot as plt
import keras.utils as image
from tqdm import tqdm
from torch.nn.functional import softmax
from glob import glob

In [ ]:
import sys
sys.path.append('/kaggle/working/lop/')
sys.path.append('/kaggle/input')

In [ ]:
from lop.algos.bp import Backprop
from lop.nets.conv_net import ConvNet
from lop.algos.convCBP import ConvCBP
from lop.nets.linear import MyLinear
from lop.utils.miscellaneous import nll_accuracy as accuracy

In [ ]:
import os
import copy
import subprocess
import itertools

In [ ]:
import torch
import pickle
import torchvision
import torchvision.transforms as transforms

def mnist():
    batch_size = 60000
    transform = transforms.Compose(
        [transforms.ToTensor()])

    train_dataset = torchvision.datasets.MNIST(
        root="data", train=True, transform=transform, download=True
    )
    test_dataset = torchvision.datasets.MNIST(
        root="data", train=False, transform=transform
    )
    # Data loader
    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=batch_size, shuffle=True, generator=torch.Generator(device='cpu')

    )
    test_loader = torch.utils.data.DataLoader(
        dataset=test_dataset, batch_size=batch_size, shuffle=False, generator=torch.Generator(device='cpu')
    )

    for i, (images, labels) in enumerate(train_loader):
        images = images.flatten(start_dim=1)
        labels = labels

    x = images
    y = labels

    for i, (images_test, labels_test) in enumerate(test_loader):
        images_test = images_test.flatten(start_dim=1)
        labels_test = labels_test

    x_test = images_test
    y_test = labels_test

    with open('data/mnist_', 'wb+') as f:
        pickle.dump([x, y, x_test, y_test], f)

    return x, y, x_test, y_test


def get_mnist(type='reg'):
    if type == 'reg':
        data_file = 'data/mnist_'
        with open(data_file, 'rb+') as f:
            x, y, x_test, y_test = pickle.load(f)
    return x, y, x_test, y_test


In [ ]:
mnist()

In [ ]:
from lop.utils.miscellaneous import get_configurations

def gen_cfg(cfg_file):

    with open(cfg_file, 'r') as f:
        params = json.load(f)

    list_params, hyper_param_settings = get_configurations(params=params)

    # make a directory for temp cfg files
    bash_command = "mkdir -p temp_cfg/"
    subprocess.Popen(bash_command.split(), stdout=subprocess.PIPE)

    bash_command = "rm -r --force " + params['data_dir']
    subprocess.Popen(bash_command.split(), stdout=subprocess.PIPE)
    bash_command = "mkdir " + params['data_dir']
    subprocess.Popen(bash_command.split(), stdout=subprocess.PIPE)

    """
        Set and write all the parameters for the individual config files
    """
    for setting_index, param_setting in enumerate(hyper_param_settings):
        new_params = copy.deepcopy(params)
        for idx, param in enumerate(list_params):
            new_params[param] = param_setting[idx]
        new_params['index'] = setting_index
        new_params['data_dir'] = params['data_dir'] + str(setting_index) + '/'

        """
            Make the data directory
        """
        bash_command = "mkdir -p " + new_params['data_dir']
        subprocess.Popen(bash_command.split(), stdout=subprocess.PIPE)

        for idx in tqdm(range(params['num_runs'])):
            new_params['data_file'] = new_params['data_dir'] + str(idx)

            """
                write data in config files
            """
            new_cfg_file = 'temp_cfg/'+str(setting_index*params['num_runs']+idx)+'.json'
            try:    f = open(new_cfg_file, 'w+')
            except: f = open(new_cfg_file, 'w+')
            with open(new_cfg_file, 'w+') as f:
                json.dump(new_params, f, sort_keys=False, indent=4)

In [ ]:
gen_cfg('/kaggle/working/lop/lop/permuted_mnist/cfg/bp/std_net.json')

In [ ]:
from lop.algos.bp import Backprop
from lop.algos.cbp import ContinualBackprop
from lop.nets.linear import MyLinear
from torch.nn.functional import softmax
from lop.nets.deep_ffnn import DeepFFNN
from lop.utils.miscellaneous import nll_accuracy, compute_matrix_rank_summaries


def online_expr(params: {}):
    agent_type = params['agent']
    num_tasks = 60
#     if 'num_tasks' in params.keys():
#         num_tasks = params['num_tasks']
#     if 'num_examples' in params.keys() and "change_after" in params.keys():
#         num_tasks = int(params["num_examples"]/params["change_after"])

    step_size = params['step_size']
    opt = params['opt']
    weight_decay = 0
    use_gpu = 0
    dev = 'cpu'
    to_log = False
    num_features = 2000
    change_after = 10 * 6000
    to_perturb = False
    perturb_scale = 0.1
    num_hidden_layers = 1
    mini_batch_size = 1
    replacement_rate = 0.0001
    decay_rate = 0.99
    maturity_threshold = 100
    util_type = 'adaptable_contribution'

    if 'to_log' in params.keys():
        to_log = params['to_log']
    if 'weight_decay' in params.keys():
        weight_decay = params['weight_decay']
    if 'num_features' in params.keys():
        num_features = params['num_features']
    if 'change_after' in params.keys():
        change_after = params['change_after']
    if 'use_gpu' in params.keys():
        if params['use_gpu'] == 1:
            use_gpu = 1
            dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
            if dev == torch.device("cuda"):    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    if 'to_perturb' in params.keys():
        to_perturb = params['to_perturb']
    if 'perturb_scale' in params.keys():
        perturb_scale = params['perturb_scale']
    if 'num_hidden_layers' in params.keys():
        num_hidden_layers = params['num_hidden_layers']
    if 'mini_batch_size' in params.keys():
        mini_batch_size = params['mini_batch_size']
    if 'replacement_rate' in params.keys():
        replacement_rate = params['replacement_rate']
    if 'decay_rate' in params.keys():
        decay_rate = params['decay_rate']
    if 'maturity_threshold' in params.keys():
        maturity_threshold = params['mt']
    if 'util_type' in params.keys():
        util_type = params['util_type']

    classes_per_task = 10
    images_per_class = 6000
    input_size = 784
    num_hidden_layers = num_hidden_layers
    net = DeepFFNN(input_size=input_size, num_features=num_features, num_outputs=classes_per_task,
                   num_hidden_layers=num_hidden_layers)

    if agent_type == 'linear':
        net = MyLinear(
            input_size=input_size, num_outputs=classes_per_task
        )
        net.layers_to_log = []

    if agent_type in ['bp', 'linear', "l2"]:
        learner = Backprop(
            net=net,
            step_size=step_size,
            opt=opt,
            loss='nll',
            weight_decay=weight_decay,
            device=dev,
            to_perturb=to_perturb,
            perturb_scale=perturb_scale,
        )
    elif agent_type in ['cbp']:
        learner = ContinualBackprop(
            net=net,
            step_size=step_size,
            opt=opt,
            loss='nll',
            replacement_rate=replacement_rate,
            maturity_threshold=maturity_threshold,
            decay_rate=decay_rate,
            util_type=util_type,
            accumulate=True,
            device=dev,
        )

    accuracy = nll_accuracy
    examples_per_task = images_per_class * classes_per_task
    total_examples = int(num_tasks * change_after)
    total_iters = int(total_examples/mini_batch_size)
    save_after_every_n_tasks = 1
    if num_tasks >= 10:
        save_after_every_n_tasks = int(num_tasks/10)

    accuracies = torch.zeros(total_iters, dtype=torch.float)
    weight_mag_sum = torch.zeros((total_iters, num_hidden_layers+1), dtype=torch.float)

    rank_measure_period = 60000
    effective_ranks = torch.zeros((int(total_examples/rank_measure_period), num_hidden_layers), dtype=torch.float)
    approximate_ranks = torch.zeros((int(total_examples/rank_measure_period), num_hidden_layers), dtype=torch.float)
    approximate_ranks_abs = torch.zeros((int(total_examples/rank_measure_period), num_hidden_layers), dtype=torch.float)
    ranks = torch.zeros((int(total_examples/rank_measure_period), num_hidden_layers), dtype=torch.float)
    dead_neurons = torch.zeros((int(total_examples/rank_measure_period), num_hidden_layers), dtype=torch.float)

    iter = 0
    with open('data/mnist_', 'rb+') as f:
        x, y, _, _ = pickle.load(f)
        if use_gpu == 1:
            x = x.to(dev)
            y = y.to(dev)

    for task_idx in (range(num_tasks)):
        new_iter_start = iter
        pixel_permutation = np.random.permutation(input_size)
        x = x[:, pixel_permutation]
        data_permutation = np.random.permutation(examples_per_task)
        x, y = x[data_permutation], y[data_permutation]

        if agent_type != 'linear':
            with torch.no_grad():
                new_idx = int(iter / rank_measure_period)
                m = net.predict(x[:2000])[1]
                for rep_layer_idx in range(num_hidden_layers):
                    ranks[new_idx][rep_layer_idx], effective_ranks[new_idx][rep_layer_idx], \
                    approximate_ranks[new_idx][rep_layer_idx], approximate_ranks_abs[new_idx][rep_layer_idx] = \
                        compute_matrix_rank_summaries(m=m[rep_layer_idx], use_scipy=True)
                    dead_neurons[new_idx][rep_layer_idx] = (m[rep_layer_idx].abs().sum(dim=0) == 0).sum()
                print('approximate rank: ', approximate_ranks[new_idx], ', dead neurons: ', dead_neurons[new_idx])

        for start_idx in tqdm(range(0, change_after, mini_batch_size)):
            start_idx = start_idx % examples_per_task
            batch_x = x[start_idx: start_idx+mini_batch_size]
            batch_y = y[start_idx: start_idx+mini_batch_size]

            # train the network
            loss, network_output = learner.learn(x=batch_x, target=batch_y)

            if to_log and agent_type != 'linear':
                for idx, layer_idx in enumerate(learner.net.layers_to_log):
                    weight_mag_sum[iter][idx] = learner.net.layers[layer_idx].weight.data.abs().sum()
            # log accuracy
            with torch.no_grad():
                accuracies[iter] = accuracy(softmax(network_output, dim=1), batch_y).cpu()
            iter += 1

        print('recent accuracy', accuracies[new_iter_start:iter - 1].mean())
        if task_idx % save_after_every_n_tasks == 0:
            data = {
                'accuracies': accuracies.cpu(),
                'weight_mag_sum': weight_mag_sum.cpu(),
                'ranks': ranks.cpu(),
                'effective_ranks': effective_ranks.cpu(),
                'approximate_ranks': approximate_ranks.cpu(),
                'abs_approximate_ranks': approximate_ranks_abs.cpu(),
                'dead_neurons': dead_neurons.cpu(),
            }
            save_data(file=params['data_file'], data=data)

    data = {
        'accuracies': accuracies.cpu(),
        'weight_mag_sum': weight_mag_sum.cpu(),
        'ranks': ranks.cpu(),
        'effective_ranks': effective_ranks.cpu(),
        'approximate_ranks': approximate_ranks.cpu(),
        'abs_approximate_ranks': approximate_ranks_abs.cpu(),
        'dead_neurons': dead_neurons.cpu(),
    }
    save_data(file=params['data_file'], data=data)


def save_data(file, data):
    with open(file, 'wb+') as f:
        pickle.dump(data, f)

In [ ]:
with open(f'temp_cfg/0.json', 'r') as f:
    params = json.load(f)

    online_expr(params)

In [ ]:
from lop.utils.miscellaneous import *
from lop.utils.plot_online_performance import generate_online_performance_plot


def add_cfg_performance(cfg='', setting_idx=0, m=2*10*1000, num_runs=30, metric='accuracy'):
    with open(cfg, 'r') as f:
        params = json.load(f)
    list_params, param_settings = get_configurations(params=params)
    per_param_setting_performance = []
    for idx in range(num_runs):
        file = params['data_dir'] + str(setting_idx) + '/' + str(idx)
        with open(file, 'rb') as f:
            data = pickle.load(f)

        if metric == 'weight':
            num_weights = 9588000
            per_param_setting_performance.append(np.array(torch.Tensor.cpu(bin_m_errs(errs=data['weight_mag_sum'].sum(dim=1)/num_weights, m=m))))
        elif metric == 'dead_neurons':
            num_units = 3*2000
            per_param_setting_performance.append(np.array(torch.Tensor.cpu(bin_m_errs(errs=data['dead_neurons'].sum(dim=1)/num_units*100, m=m))))
        elif metric == 'effective_rank':
            rank_normlization = 3*2000/100
            per_param_setting_performance.append(np.array(torch.Tensor.cpu(bin_m_errs(errs=data['effective_ranks'].sum(dim=1)/rank_normlization, m=m))))
        else:
            per_param_setting_performance.append(np.array(torch.Tensor.cpu(bin_m_errs(errs=data['accuracies'] * 100, m=m))))
    print(param_settings[setting_idx], setting_idx, np.array(per_param_setting_performance).mean())
    return np.array(per_param_setting_performance)

def bp_metrics(cfg_file, metric):
    with open(cfg_file, 'r') as f:
        params = json.load(f)
    list_params, param_settings = get_configurations(params=params)

    performances = []
    m = {'weight': 60*1000, 'accuracy': 60*1000, 'dead_neurons': 1, 'effective_rank': 1}[metric]
    num_runs = 1

    indices = [i for i in range(1)]
    for i in indices:
        performances.append(add_cfg_performance(cfg=cfg_file, setting_idx=i, m=m, num_runs=num_runs, metric=metric))
    
    print(performances)

    yticks = {'weight': [0, 0.02, 0.04, 0.06, 0.08, 0.10], 'accuracy': [88, 90, 92, 94, 96],
              'dead_neurons': [0, 10, 20, 30], 'effective_rank': [0, 10, 20, 30, 40, 50]}[metric]
    generate_online_performance_plot(
        performances=performances,
        colors=['C1', 'C3', 'C5', 'C2', 'C4', 'C6'],
        yticks=yticks,
        xticks=[0, 15*m, 30*m, 45*m, 60*m],
        xticks_labels=['0', '15', '30', '45', '60'],
        m=m,
        fontsize=18,
        labels=param_settings,
    )

In [ ]:
bp_metrics('/kaggle/working/lop/lop/permuted_mnist/cfg/bp/std_net.json', 'accuracy')

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
graph = np.asarray(Image.open('comparison.png'))
plt.imshow(graph)

In [ ]:
gen_cfg('/kaggle/working/lop/lop/permuted_mnist/cfg/bp/small_net.json')
with open(f'temp_cfg/0.json', 'r') as f:
    params = json.load(f)

    online_expr(params)

In [ ]:
bp_metrics('/kaggle/working/lop/lop/permuted_mnist/cfg/bp/small_net.json', 'accuracy')

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
graph = np.asarray(Image.open('comparison.png'))
plt.imshow(graph)

In [ ]:
gen_cfg('/kaggle/working/lop/lop/permuted_mnist/cfg/adam.json')
with open(f'temp_cfg/0.json', 'r') as f:
    params = json.load(f)

    online_expr(params)

In [ ]:
bp_metrics('/kaggle/working/lop/lop/permuted_mnist/cfg/adam.json', 'accuracy')

In [ ]:
graph = np.asarray(Image.open('comparison.png'))
plt.imshow(graph)

In [ ]:
!mkdir data/cbp
!mkdir data/cbp/0

In [ ]:
cfg = {
    "model_version": "CBP on net with 3 hidden layer and 2000 hidden-units",
    "num_runs":  1,
    "num_tasks": 60,
    "use_gpu": 1,
    "data_dir": "data/cbp/0/",
    "to_log": True,

    "agent": "cbp",
    "opt": "sgd",
    "num_features": 500,
    "num_hidden_layers": 3,
    "step_size": 0.003,
    "accumulate": True,
    "util_type": "adaptable_contribution",
    "replacement_rate": 1e-4,
    "decay_rate": 0.99,
    "mt": 100,
    "index": 0,
    "data_file": "data/cbp/0/0"
}
online_expr(cfg)

In [ ]:
!zip -r res.zip /kaggle/working/data/bp

In [ ]:
from IPython.display import FileLink

FileLink(r'res.zip')